# Band A Vectors over the RGB Image created using CO Map from Alma data and H2 Map data from Bally et al.

#### Making rgb cube using aplpy

In [ ]:
import aplpy as ap
ap.make_rgb_cube(['max_high_V.fits',
                  'OrionH2_NICFPS_cropped.fits',
                  'OrionH2_NICFPS_cropped.fits'], 'CO_H2_Fe1.fits')

#### Making rgb image using the rgb cube and stretching it the blue and green to log scale

In [ ]:
ap.make_rgb_image('CO_H2_Fe1.fits', 'CO_H2_Fe1.png',
                  stretch_b = 'log', stretch_g = 'log'
                  ,vmin_b=100, vmax_b=20000.0
                  ,vmin_g = 100, vmax_g=20000.0)

#### Importing all the necessary modules

In [ ]:
import aplpy as ap
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import wcs
import matplotlib.image as mpimg
import numpy as np
from astropy.coordinates import SkyCoord
import matplotlib.colors

#### Importing Band A file and 2d fits file made by aplpy to project as the map with common resolution for all three files

In [ ]:
file1 = 'CO_H2_Fe1_2d.fits'
bandA = 'F0454_HA_POL_unk3_HAWAHWPA_PMP_013-145.fits'

#read in image
fits_file1 = fits.open(file1)
data1=fits_file1[0].data
w1=wcs.WCS(fits_file1[0].header) #wcs information

# read the band A file
polfile=fits.open(bandA)
wa=wcs.WCS(polfile[0].header)
xpix=polfile[0].header['NAXIS1']
ypix=polfile[0].header['NAXIS2']
polfile[0].header['CDELT2']

#### Making the figure, setting up coordinates, and showing the rgb image

In [ ]:
plt.figure(figsize=(8,8))
ax=plt.axes(projection=w1)
ra=ax.coords[0]
dec=ax.coords[1]
ra.display_minor_ticks(True)
dec.display_minor_ticks(True)
ra.set_major_formatter('hh:mm:ss.s')

# reading the rgb image file and showing it in the axes
image = mpimg.imread("CO_H2_Fe1.png")
ax.imshow(np.flipud(image))#, origin='lower')

#### Extracting polangle and polarization information from Band A file for plotting vectors

In [ ]:
skip=2
#set polarizations and angles to nan if zero
pol, pol_err, ang = polfile['PERCENT POL'].data, polfile['ERROR PERCENT POL'].data, polfile['POL ANGLE'].data
pol[pol == 0.] = np.nan
ang[pol == 0.] = np.nan

#set low sn values to nan
pol[pol/pol_err<3]=np.nan
pol[21,21] = 10.
ang[20,20] = 0.
aa = pol[~np.isnan(pol)]
aa[aa > 0]
x, y = np.meshgrid(np.arange(pol.shape[1]), np.arange(pol.shape[0]))

# mask to check the number of vectors being plotted
mask=np.where((x%skip+y%skip)==0)
pol=pol[mask]
ang=ang[mask]
x=x[mask]
y=y[mask]
ra, dec = wa.wcs_pix2world(x, y, 0)
x1,y1=w1.wcs_world2pix(ra,dec,0)

#Coords required for quiver
xx = pol*np.cos(ang*np.pi/180.)
yy = pol*np.sin(ang*np.pi/180.)

# setting the coordinated of the axes and limiting them
lowerleft= SkyCoord("5h35m20.865s -5d23m37s")
upperright=SkyCoord("5h35m8.27s -5d20m7.94s")
lx,ly=w1.wcs_world2pix(lowerleft.ra.deg,lowerleft.dec.deg,0)
ux,uy=w1.wcs_world2pix(upperright.ra.deg,upperright.dec.deg,0)
ax.set_xlim(lx,ux)
ax.set_ylim(ly,uy)

#### Making the vectors black and white based on the intensity of background

In [ ]:
list_color = []
for i in range(len(x1)):
    try:
        if data1[int(y1[i]), int(x1[i])] > 1600: 
            list_color.append('k')
        else:
            list_color.append('w')
    except Exception as e:
        list_color.append('k')  

#### Plotting the vectors

In [ ]:
ax.quiver(x1,y1,xx,yy,angles='xy', pivot='middle',
          headwidth=1,headlength=0,headaxislength=0,
          linewidths=0.5,scale_units='width',width=0.001,scale=500., color=list_color)
          
# saving the fig in high resolution
plt.savefig('CO_H2_Fe_vec_plotted_black_white.png', dpi=360)